<a href="https://colab.research.google.com/github/10100111/Display-of-HW1/blob/main/61_Lite_%D0%A1%D0%B5%D0%B3%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D1%8F_%D0%A2%D0%B5%D0%BA%D1%81%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание Lite

Макс 10 баллов

1. Сделайте свою собственную архитектуру и запустите, оцените точность на проверочной выборке.

2. Возьмите лучшую архитектуру и проведите несколько экспериментов, меняя гиперпараметры выбранной архитектуры.

# Сегментация текстов

# Импорт библиотек

https://youtu.be/wdi8HYhgGj0?t=97

In [ ]:
!pip install -q pymorphy2 # установка лемматизатора pymorphy2

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 10.1 MB/s 


In [ ]:
import numpy as np # Библиотека для работы с массивами данных
from tensorflow.keras.models import Model, load_model # Импортируем Model, load_model - метод, что загружает предобученную сеть
import re # Имортируем чтобы работать с строками
from tensorflow.keras.preprocessing.text import Tokenizer # Метод, который поволяет работать с текстами и конвертирует их в последовательности (индексов)
# Импорт слоёв нейронных сетей
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Conv1DTranspose, Lambda
from tensorflow.keras import backend as K # Импортируем, чтобы высчитать dice_coef(ошибку)
from tensorflow.keras.optimizers import Adam, Adadelta # Импортируем оптимизаторы
from tensorflow.keras import utils # Импортируем для работы с категориальными данными
from tensorflow.keras.utils import plot_model
import tensorflow
from google.colab import files # Импорт для работы с файлами
import matplotlib.pyplot as plt # Импорт для отрисовывания графиков
from gensim.models import word2vec # Импортируем gensim
import os # Импортируем для работы с системными файлами
import pandas as pd # Импортируем для работы с Массивами данных(таблицами - Датафреймами)
import time # Имортируем, чтобы высчитать время работы каких-либо процессов
import nltk #Natural language toolkit - Инструментарий естественного языка
from nltk.stem import WordNetLemmatizer  # Импортируем для работы с леммами
import pymorphy2 # Импортируем для работы с леммами

nltk.download('wordnet') # Скачиваем сетку слов для лемматизации

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Функции для работы с текстами

https://youtu.be/wdi8HYhgGj0?t=126

Превращаем договоры в листы слов

# **Лист тегов и их значений**
https://youtu.be/wdi8HYhgGj0?t=48
**`<s1>`** - Условия

**`<s2>`** - Запреты

**`<s3>`** - Цена

**`<s4>`** - Условия (относительно дат)

**`<s5>`** - Штрафы

**`<s6>`** - Адреса и местоположения


In [ ]:
# Считываем текст из файла и удаляем пунктуационные знаки препинания и еще дополнительные ненужные нам знаки
def readText(fileName):
  f = open(fileName, 'r') # Открываем наш файл и считываем наши данные
  text = f.read() # Помещаем наш текст в переменную
  # Определяем, какие знаки будут удалены
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";",'•','%']

  for dS in delSymbols: # Каждый знак из нашего списка будет удалён из списка
    text = text.replace(dS, " ") # Удаляем знак, посредством замены этого знака на пробел

  # Выискиваем дополнительные знаки в текстах посредством паттернов и определяем на что их заменять
  text = re.sub("[.]", " ", text)
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

  # split  метод разделит текст на последовательность слов при помощи пробелов
  # (а их может быть в одном месте очень много после наших удалений знаков), которые будет являться разделителями.
  # Используя join метод, мы обратно соберём последовательности слов в текст
  text = ' '.join(text.split()) 

  text = text.lower() # Конвертируем текст к нижнему регистру
  return text # Возвращаем текст

# Конвертируем исходный текст в лист слов с начальной формой 
def text2Words(text):
  morph = pymorphy2.MorphAnalyzer() # Инициализируем инструмент для работы с морфемами и более
  words = text.split(' ') #  Разделяем текст по средством пробелов
  docs = [morph.parse(word)[0].normal_form for word in words] # Превращаем каждое слово в элемент списка
  return docs # Возвращаем полученный документ

# Read the data

https://youtu.be/wdi8HYhgGj0?t=330



In [ ]:
directory = '/content/drive/MyDrive/Базы/Договоры 432/' # Путь к папке с договорами
os.listdir(directory)[250:255] # Выводим на экран название 5 договоров

['dogovor-zaloga-s-peredachej-predmeta-zaloga-zalogoderzhatelyu.txt',
 'dogovor-zaloga-privatizirovannoy-kvartiri.txt',
 'dogovor-zaloga-imuschestvennih-prav.txt',
 'dogovor-zaloga-imuschestva.txt',
 'dogovor-zaloga-imuschestva-kvartiri.txt']

In [ ]:
print('Всего', len(os.listdir(directory)), 'договора')

Всего 432 договора


In [ ]:
curTime = time.time() # Запоминаем текущее время
agreements = [] # Лист, что будет хранить все наши договоры
for filename in os.listdir(directory): # Проходимся по каждому договору
  txt = readText(directory + filename) # Считываем текст в договоре
  if txt != '': # Если договор не пустой
    agreements.append(readText(directory + filename)) # Превращаем файл в текст и добавляем его в agreements
print('Загрузка файла заняла: ', round(time.time() - curTime, 2), 'с.')

Загрузка файла заняла:  3.36 с.


In [ ]:
n = 3
print(os.listdir(directory)[n]) # Выводим названия n-ого договора
agreements[n]                   # Текст n-ого договора

2 (2).txt


'договор благотворительного пожертвования целевое использование г г в лице действующего на основании именуемый в дальнейшем благотворитель с одной стороны и в лице действующего на основании именуемый в дальнейшем благополучатель с другой стороны именуемые в дальнейшем стороны заключили настоящий договор в дальнейшем договор о нижеследующем 1 общие положения 1 1 <s3> благотворитель передает благополучателю денежные средства благотворительное пожертвование для реализации благотворительной программы </s3> 1 2 цели программы 1 3 программа и бюджет прилагаются к договору и являются неотъемлемой частью настоящего договора 1 4 <s4> настоящий договор начинает действовать с года датой окончания договора является года настоящий договор вступает в силу с момента заключения и заканчивается после выполнения принятых сторонами на себя обязательств в соответствии с условиями договора </s4> 2 порядок предоставления и расходования пожертвования 2 1 <s3> общая сумма благотворительного пожертвования сост

In [ ]:
docs_full = [] # Лист будет содержать все договоры как списки документов
curTime = time.time() # Запоминаем текущее время
for i in range(len(agreements)): # Проходимся по каждому договору
  docs_full.append(text2Words(agreements[i])) # Превращаем договор в лист слов и добавляем его в docs_full
print('Превращение заняло: ', round(time.time() - curTime, 2), 's.')

Превращение заняло:  182.7 s.


In [ ]:
print(len(docs_full)) # Выводим число записей в наборе данных 

# Выбираем итоговое количество данных для обучающей/проверочной и тестовой выборках
docs = docs_full[0:-10]
docsToTest = docs_full[-10:]

422


In [ ]:
print('Число текстов в для тестовой проверки в конце ноутбука:', len(docsToTest)) 
print('Число договоров для обучающей и проверочной выборках:',len(docs)) 
print('Число слов в первом договоре:', len(docs[10]))

Число текстов в для тестовой проверки в конце ноутбука: 10
Число договоров для обучающей и проверочной выборках: 412
Число слов в первом договоре: 2376


In [ ]:
print('Пример текста обычного:')
print(agreements[4][:62], '\n')
print('Тот же текст, но представленный ввиде лемм:')
print(docs[4][:10])

Пример текста обычного:
договор бартера г г в лице действующего на основании именуемый 

Тот же текст, но представленный ввиде лемм:
['договор', 'бартер', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать']


# Превращение текстов в последовтельность индексов: создание xTrain
https://youtu.be/wdi8HYhgGj0?t=594

In [ ]:
# lower=True - приводит к нижнему регистру все слова
# char_level=False - говорит токенайзеру не инициализировать отдельно каждую букву как токен  
tokenizer = Tokenizer(lower=True, filters = '', char_level=False)

tokenizer.fit_on_texts(docs_full) # Скармливаем тексты токенайзеру
clean_voc = {}                    # Создаем пустой словарь

for item in tokenizer.word_index.items(): # Превращаем лист в словарь
  clean_voc[item[0]] = item[1]            # Мы меняем местами элеметны кортежа

In [ ]:
print('Словарь все слов по их частотноти:') 
print(clean_voc, '\n')
print('Длина словаря:', len(clean_voc))

tag = '<s1>'
print('Индекс тега', tag, ':' ,clean_voc[tag])

Словарь все слов по их частотноти:
{'в': 1, 'договор': 2, 'и': 3, '2': 4, '1': 5, 'по': 6, 'сторона': 7, 'с': 8, 'настоящий': 9, 'на': 10, '3': 11, '<s1>': 12, '</s1>': 13, 'не': 14, '4': 15, 'или': 16, 'за': 17, 'он': 18, 'случай': 19, 'о': 20, '5': 21, '<s3>': 22, '</s3>': 23, 'право': 24, 'работа': 25, 'срок': 26, '<s4>': 27, '</s4>': 28, 'от': 29, 'при': 30, 'они': 31, '6': 32, 'другой': 33, 'быть': 34, 'лицо': 35, 'имущество': 36, 'день': 37, 'действовать': 38, 'арендатор': 39, 'условие': 40, 'для': 41, '7': 42, 'к': 43, 'если': 44, 'п': 45, 'дальнейший': 46, 'работник': 47, 'который': 48, 'а': 49, 'обязательство': 50, 'страховой': 51, 'законодательство': 52, 'порядок': 53, 'сумма': 54, 'размер': 55, 'действие': 56, 'арендодатель': 57, 'свой': 58, 'счёт': 59, 'соответствие': 60, 'указанный': 61, 'именовать': 62, 'мочь': 63, 'из': 64, 'основание': 65, 'обязанность': 66, 'обязать': 67, 'исполнение': 68, 'один': 69, 'страховщик': 70, 'страхователь': 71, 'иной': 72, 'год': 73, 'до': 7

In [ ]:
# Превращает текст в последовательность индексов согласно словарю частотности
tok_agreem = tokenizer.texts_to_sequences(docs) # Обучающий выборка в последовтельность индексов

print("Взгляните на фрагмент обучающей выборки:")
print("Исходный текст:              ", docs[4][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreem[4][:20], '\n')

Взгляните на фрагмент обучающей выборки:
Исходный текст:               ['договор', 'бартер', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'сторона', '1', 'с', 'один', 'сторона', 'и', 'в', 'лицо']
Тот же текст, но как последовательность индексов:  [2, 4233, 76, 76, 1, 35, 38, 10, 65, 62, 1, 46, 7, 5, 8, 69, 7, 3, 1, 35] 



# Создание yTrain

https://youtu.be/wdi8HYhgGj0?t=695

Каждый индекс (слово) из договора будет соответсвовать вектору, чьи элементы соотвествуют категориям тегов

In [ ]:
#  Собираем лист индексов и их мультилейбл класссификации
def getXYSamples(tok_agreem, tags_index):
  tags01 = [] # лист тегов
  indices = [] # лист индексов
 
  for agreement in tok_agreem: # Проходимся по каждому договору-листу
    tag_place = [0, 0, 0, 0, 0, 0] # создаём вектор [0,0,0,0,0,0] 
    for ex in agreement: # Проходимся по каждому слову из договора
        if ex in tags_index: # Проверяем, если индекс слова соотвествует индексу одного из тегов
          place = np.argwhere(tags_index==ex) # Запоминаем позицию тега в этом листе тегов
          if len(place)!=0: # Проверяем, если тек в самом деле есть
            if place[0][0]<6: # Первые 6 тегов - открывающие теги
              tag_place[place[0][0]] = 1    # устанавливаем значение 1
            else: 
              tag_place[place[0][0] - 6] = 0  # Все остальные теги закрывающие, так что мы меняем в соответсвующем месте в векторе на 0
        else:          
          tags01.append(tag_place.copy()) # Добавляем в наш лист тегов новые вектора мульти-лейбл классификации. 
                                          # В конце концов у нас будет большой лист всех вектор-тегов
          indices.append(ex) # Добавляем индекс-слова в лист индексов

  return indices, tags01

# Получаем лист слов из листа индексов
def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) #  Берёт ключ и значение из словаря и меняет их местами
  docs = [reverse_word_map.get(letter) for letter in x] # Положим их в лист
  return docs # Возвращает текст

In [ ]:
#<s1> - Условия договора
#<s2> - Запреты
#<s3> - Стоимость
#<s4> - Условия (относительно дат)
#<s5> - Штрафы
#<s6> - Адреса и местоположения

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] # Лист открывающих тегов
closetags = ['</s' + str(i) + '>' for i in range(1, 7)] # Лист закрывающих тегов
tags_index.extend(closetags) # конкатенируем все теги

tags_index = np.array([clean_voc[i] for i in tags_index]) # Получаем инексы всех тегов из словря частотности
print('Индексы всех тегов:', tags_index)

Индексы всех тегов: [ 12 341  22  27 117 278  13 352  23  28 118 273]


In [ ]:
curTime = time.time()
xData, yData = getXYSamples(tok_agreem,tags_index) # Получаем теги и создаём листы с ними
decoded_text = reverseIndex(clean_voc, xData) # Чтобы создать лист для эмбеддинга, нам нужно превратить лист индексов обратно в лист слов
print('Превращение заняло: ', round(time.time() - curTime, 2), 'с.')

Превращение заняло:  2.61 с.


In [ ]:
print('длина xData:', len(xData))
print('длина yData:', len(yData))

print('Посмотрим на исходные слова: ', docs[0][50:80])
print('Сдекодированые слова:', decoded_text[50:80])
print('Часть xData:     ', xData[50:80])
print('Часть yData:     ', yData[50:80])

длина xData: 409700
длина yData: 409700
Посмотрим на исходные слова:  ['1', 'предмет', 'договор', '1', '1', '<s1>', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '</s1>', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по']
Сдекодированые слова: ['1', 'предмет', 'договор', '1', '1', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор']
Часть xData:      [5, 112, 2, 5, 5, 6, 9, 2, 224, 115, 918, 319, 125, 18, 250, 3, 546, 244, 319, 1, 396, 5, 4, 250, 304, 10, 162, 6, 9, 2]
Часть yData:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0,

# Разделение выборки на окна

https://youtu.be/wdi8HYhgGj0?t=1310

In [ ]:
# Создание выборки из индексов
def getSetFromIndices(wordIndices, xLen, step): 
  xBatch = [] # Лист, хранящий фрагменты текста
  wordsLen = len(wordIndices) # получаем длину текста
  index = 0 #  стартовый индекс
  
  while (index + xLen <= wordsLen): # Пока сумма индекса и окна меньше или равно чем число слов в выборке
    xBatch.append(wordIndices[index:index+xLen]) # Добавляем X в лист, что содержит наши фрагменты(окна) текста
    index += step # Смещаем наше индекс на наш шаг(step)

  return xBatch #  Лист фрагментов текста

In [ ]:
xLen = 256 # Длина каждого окна
step = 30 # шаг 
embeddingSize = 300 # Число измерений вектор-пространства для каждого нашего слова 

In [ ]:
#  Генерируем выборки с параметрами наших окон
xTrain = getSetFromIndices(decoded_text, xLen, step) # Последовательность индексов с окном xLen слов
yTrain = getSetFromIndices(yData, xLen, step) # Последовательность индексов с окном xLen от тегов

In [ ]:
print('Длина xTrain:', len(xTrain))
print('Длина yTrain:', len(yTrain))
print('Длина примера из xTrain:',len(xTrain[0]))
print('Длина примера из yTrain:',len(yTrain[0]), '\n')
print('Пример xTrain', xTrain[0])
print('Пример yTrain', yTrain[0], '\n')

print('Первый пример xTrain:', xTrain[0][step-5:step+5])
print('Второй пример xTrain:', xTrain[1][:10])

Длина xTrain: 13649
Длина yTrain: 13649
Длина примера из xTrain: 256
Длина примера из yTrain: 256 

Пример xTrain ['договор', 'безвозмездный', 'хранение', 'до', 'востребование', 'г', 'г', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'хранитель', 'с', 'один', 'сторона', 'и', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'поклажедатель', 'с', 'другой', 'сторона', 'именовать', 'в', 'дальнейший', 'сторона', 'заключить', 'настоящий', 'договор', 'в', 'дальнейший', 'договор', 'о', 'нижеследующий', '1', 'предмет', 'договор', '1', '1', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор', 'следующий', 'вещь', 'далее', 'вещь', '1', '3', 'при', 'передача', 'вещь', 'по

In [ ]:
# Превращаем текст в последовательность индексов согласно словарю частотности
tok_agreemTest = tokenizer.texts_to_sequences(docsToTest) # Обучающий текст в индексы

print("Посмотрим на фрагмент тестового текста:")
print("Исходный текст:              ", docsToTest[4][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreemTest[4][:20], '\n')

xDataTest, yDataTest = getXYSamples(tok_agreemTest,tags_index) # Распознаём теги и создаём лист, что их хранят
decoded_text = reverseIndex(clean_voc, xDataTest) # Чтобы создать лист для эмбеддинга, нам нужно превратить лист индексов обратно в лист слов
print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))

print('Сдекодированные текст:', decoded_text[50:80])
print('Часть xDataTest:     ', xDataTest[50:80])
print('Часть yDataTest:     ', yDataTest[50:80])

xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Число измерений вектор-пространства для каждого нашего слова 

# Генерируем выборки с определёнными параметрами
xTest = getSetFromIndices(decoded_text, xLen, step) # Последовательность индексов с окном Xlen слов
yTest = getSetFromIndices(yDataTest, xLen, step) # # Последовательность индексов с окном Xlen от тегов

print('Длина xTest:', len(xTest))
print('Длина yTest:', len(yTest))
print('Длина примера xTest:',len(xTest[0]))
print('Длина примера yTrain:',len(yTest[0]), '\n')
print('Пример xTest', xTest[0])
print('Пример yTest', yTest[0], '\n')

print('Первый пример xTest:', xTest[0][step-5:step+5])
print('Второй пример xTest:', xTest[1][:10])

Посмотрим на фрагмент тестового текста:
Исходный текст:               ['трудовой', 'контракт', 'с', 'руководитель', 'кадровый', 'служба', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'общество', 'с', 'один', 'сторона']
Тот же текст, но как последовательность индексов:  [79, 187, 8, 675, 4217, 640, 76, 76, 1, 35, 38, 10, 65, 62, 1, 46, 146, 8, 69, 7] 

Длина xDataTest: 11750
Длина yDataTest: 11750
Сдекодированные текст: ['положение', '1', '1', 'настоящий', 'договор', 'заключить', 'между', 'в', 'лицо', 'председатель', 'совет', 'директор', 'именовать', 'в', 'дальнейший', 'работодатель', 'с', 'один', 'сторона', 'и', 'гражданин', 'россия', 'именовать', 'в', 'дальнейший', 'директор', '1', '2', 'договор', 'регулировать']
Часть xDataTest:      [209, 5, 5, 9, 2, 109, 182, 1, 35, 1902, 537, 217, 62, 1, 46, 129, 8, 69, 7, 3, 265, 941, 62, 1, 46, 217, 5, 4, 2, 1060]
Часть yDataTest:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0,

# Функции создания моделей
https://youtu.be/wdi8HYhgGj0?t=1574

## Функция, что считает ошибку
https://youtu.be/wdi8HYhgGj0?t=1566

In [ ]:
# Функция нужна для точности
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

## **Conv1D**

https://youtu.be/wdi8HYhgGj0?t=1635

In [ ]:
# Функция, что создаёт линейную свёрточную сеть
def create_Conv1d(k, xLen, embeddingSize): 
  text_input_layer = Input((xLen,embeddingSize)) 
  text_layer = Conv1D(16 * k, 3, padding='same',activation='relu')(text_input_layer)
  text_layer = BatchNormalization()(text_layer)
  text_layer = Dropout(0.25)(text_layer) 
  text_layer = Conv1D(16 * k, 3, padding='same',activation='relu')(text_layer)
  text_layer = BatchNormalization()(text_layer)
  text_layer = Dropout(0.25)(text_layer)
  text_layer = Conv1D(16 * k, 3,padding='same', activation='relu')(text_layer)
  text_layer = BatchNormalization()(text_layer) 
  text_layer = Dropout(0.25)(text_layer)
  text_layer = Conv1D(16 * k, 3,padding='same', activation='relu')(text_layer)
  text_layer = BatchNormalization()(text_layer) 
  text_layer = Dropout(0.25)(text_layer)
  text_layer = Conv1D(16 * k, 3,padding='same', activation='relu')(text_layer)
  text_layer = BatchNormalization()(text_layer) 
  text_layer = Dropout(0.25)(text_layer)
  text_layer = Conv1D(16 * k, 3,padding='same', activation='relu')(text_layer) 
  text_layer = BatchNormalization()(text_layer)
  text_layer = Dropout(0.25)(text_layer)
  text_layer = Conv1D(16 * k, 3,padding='same', activation='relu')(text_layer) 
  text_layer = Conv1D(GENSIMtrainY.shape[-1], 3, padding='same',activation='sigmoid')(text_layer)
  model = Model(text_input_layer, text_layer)
  model.compile(optimizer=Adadelta(),
                    loss='categorical_crossentropy',
                    metrics=[dice_coef])
  return model

# Создание xTrain и  yTrain используя Word2Vec GENSIM

https://youtu.be/wdi8HYhgGj0?t=1830


In [ ]:
def pad_zeros(phrase, xLen = 256): # Функция для дополнением нулями
  while len(phrase) < xLen: # Пока сущестующий размер меньше чем требуемый 
    phrase.append([0] * embeddingSize) # Продолжаем добавлять элементы(нули) в фразу(zero-pad)  
  if len(phrase) > xLen: # В противном случае, если фраза больше чем  требуемый размер
    phrase = phrase[:xLen] # Берём только требуемое число элементов
  return phrase # Возвращаем обработанную фразу

# Создаём выборку
def getSets(model, senI, tagI):
  xVector = [] # Лист будет содержать ембеддинг представление на каждый индекс
  tmp = [] # Временный список
  for text in senI: # Проходимся по каждому текту-списку
    tmp=[]
    for word in text: # Проходимся по кажому слову тексте-списке
      try: # Если слово есть в модели (словарь не слишком мал), мы не получим ошибку
        tmp.append(model.wv[word])
      except: # В противном случае пропустим слово
        pass

    xVector.append(pad_zeros(tmp, xLen))
  temp = np.array(xVector)
  return np.array(xVector, dtype = np.float32), np.array(tagI)

In [ ]:
# Подаём лист листов слов в word2vec для обучающей выборке
# size = embeddingSize - Размер ембеддинга
# window = 10 -  минимальное расстояния между эмбеддинг словами
# min_count = 1 - Игнорируем все слова с частотой меньше чем 1
# workers = 10 -  Число потоков на обучение эмбеддинга
# iter = 20 -  Число эпох на обучение эмбеддинга
# max_vocab_size = 1e5 -  Число слов в "словарном запасе" word2vec
curTime = time.time()

modelGENSIM = word2vec.Word2Vec(xTrain + xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10, max_vocab_size = 1e5)
print('Обучение GENSIM модели заняло: ', round(time.time() - curTime, 2), 'с.')

Обучение GENSIM модели заняло:  15.92 с.


In [ ]:
modelGENSIM.wv['договор'] # Пример превращения слова в вектор при помощи GENSIM

array([-1.43768823e+00,  9.91074741e-01, -1.79073083e+00, -1.39338225e-01,
       -1.85171866e+00,  8.49334002e-01, -9.05990601e-01, -2.64716649e+00,
       -5.30157685e-01, -2.34895468e+00,  3.49125773e-01,  4.10982966e-01,
        3.66071254e-01, -1.74924946e+00,  4.27256525e-01, -6.20796144e-01,
        1.73909500e-01,  2.42911482e+00,  2.95079470e-01,  9.66953516e-01,
        1.23421825e-01, -5.30592978e-01, -1.23355818e+00,  3.58276367e-01,
       -1.28330028e+00,  9.34713364e-01, -2.83089495e+00, -1.03664450e-01,
        5.32860756e-01,  1.46670365e+00, -1.50056291e+00, -5.52783906e-01,
        1.82546329e+00,  4.03072476e-01, -3.45024824e-01,  4.36556160e-01,
        1.22041106e+00,  2.55771375e+00,  2.19808483e+00, -1.15011919e+00,
       -1.15520477e+00, -4.03426677e-01, -4.38949585e+00,  2.30968165e+00,
       -6.04066491e-01, -3.96102900e-03, -5.01374185e-01,  9.50974107e-01,
        2.40910029e+00,  1.00165939e+00,  1.22196949e+00,  2.11343002e+00,
        1.33514211e-01, -

In [ ]:
GENSIMtrainX, GENSIMtrainY = getSets(modelGENSIM, xTrain, yTrain)
GENSIMtestX, GENSIMtestY = getSets(modelGENSIM, xTest, yTest)

In [ ]:
print('Размерности xTrain:', GENSIMtrainX.shape)
print('Размерности yTrain:', GENSIMtrainY.shape)

Размерности xTrain: (13649, 256, 300)
Размерности yTrain: (13649, 256, 6)


# Train the networks
https://youtu.be/wdi8HYhgGj0?t=2099

## **Linear Conv1d network**

https://youtu.be/wdi8HYhgGj0?t=2103

In [ ]:
model_conv1d = create_Conv1d(1, xLen, embeddingSize) # Создаем модель
model_conv1d.summary() # Выводим саммери

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 300)]        0         
                                                                 
 conv1d (Conv1D)             (None, 256, 16)           14416     
                                                                 
 batch_normalization (BatchN  (None, 256, 16)          64        
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256, 16)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 256, 16)           784       
                                                                 
 batch_normalization_1 (Batc  (None, 256, 16)          64        
 hNormalization)                                             

In [ ]:
history = model_conv1d.fit(GENSIMtrainX, GENSIMtrainY, epochs=20, batch_size=200, validation_data=(GENSIMtestX, GENSIMtestY)) # Обучаем модель на сверточной сети

Epoch 1/20
69/69 [==============================] - 11s 123ms/step - loss: 1.6410 - dice_coef: 0.1889 - val_loss: 1.2819 - val_dice_coef: 0.1893
Epoch 2/20
69/69 [==============================] - 6s 93ms/step - loss: 1.6382 - dice_coef: 0.1893 - val_loss: 1.3268 - val_dice_coef: 0.1867
Epoch 3/20
69/69 [==============================] - 7s 97ms/step - loss: 1.6354 - dice_coef: 0.1892 - val_loss: 1.3558 - val_dice_coef: 0.1863
Epoch 4/20
69/69 [==============================] - 7s 97ms/step - loss: 1.6308 - dice_coef: 0.1897 - val_loss: 1.3741 - val_dice_coef: 0.1864
Epoch 5/20
69/69 [==============================] - 7s 96ms/step - loss: 1.6278 - dice_coef: 0.1898 - val_loss: 1.3842 - val_dice_coef: 0.1866
Epoch 6/20
69/69 [==============================] - 7s 97ms/step - loss: 1.6248 - dice_coef: 0.1898 - val_loss: 1.3884 - val_dice_coef: 0.1869
Epoch 7/20
69/69 [==============================] - 6s 94ms/step - loss: 1.6203 - dice_coef: 0.1901 - val_loss: 1.3891 - val_dice_coef: 0.18

In [ ]:
k = [2, 4, 8, 16, 32, 64]         # коэффициент количества нейронов в свертках
batch_size = [64, 128, 256, 512]  # значения размеров batch_size
# проведем в цикле исследование на лучший набор гиперпараметров
for coefficient in k:             # для каждого коэффициента 16 * k (множитель в слоях в функции сборки модели)
  for bs in batch_size:           # подберем батч сайз
    model_conv1d = create_Conv1d(coefficient, xLen, embeddingSize)  # создадим модель
    history = model_conv1d.fit(GENSIMtrainX, GENSIMtrainY, epochs=50, batch_size=bs, validation_data=(GENSIMtestX, GENSIMtestY),verbose=0) # запустим обучение с набором параметров 
    # выведем на печать результат каждого запуска цикла
    print('Количество нейронов в свертках = ', coefficient*16)
    print('Batch_size = ', bs)
    print('val_loss:',  round(history.history['val_loss'][-1], 4), 'val_dice_coef:', round(history.history['val_dice_coef'][-1], 4)) 


Количество нейронов в свертках =  32
Batch_size =  64
val_loss: 0.9756 val_dice_coef: 0.257
Количество нейронов в свертках =  32
Batch_size =  128
val_loss: 1.0688 val_dice_coef: 0.2248
Количество нейронов в свертках =  32
Batch_size =  256
val_loss: 1.1069 val_dice_coef: 0.2135
Количество нейронов в свертках =  32
Batch_size =  512
val_loss: 1.2683 val_dice_coef: 0.1993
Количество нейронов в свертках =  64
Batch_size =  64
val_loss: 0.8854 val_dice_coef: 0.2811
Количество нейронов в свертках =  64
Batch_size =  128
val_loss: 1.0166 val_dice_coef: 0.2523
Количество нейронов в свертках =  64
Batch_size =  256
val_loss: 1.0755 val_dice_coef: 0.2173
Количество нейронов в свертках =  64
Batch_size =  512
val_loss: 1.2299 val_dice_coef: 0.2173
Количество нейронов в свертках =  128
Batch_size =  64
val_loss: 1.326 val_dice_coef: 0.3209
Количество нейронов в свертках =  128
Batch_size =  128
val_loss: 0.959 val_dice_coef: 0.226
Количество нейронов в свертках =  128
Batch_size =  256
val_loss:

# **Проверяем работу сети**

https://youtu.be/wdi8HYhgGj0?t=2244

In [ ]:
def recognizeSet(XX, YY, model, tags, length, value):
  correct_list = np.array([0] * 6) #Инициализируем массив правильных ответов в нули (сколько раз правильно определили класс)
  incorrect_list =  np.array([0] * 6)  #Инициализируем массив неправильных ответов в нули  (сколько раз неправильно определили класс)
  XX_array = XX
  YY_array = YY
  pred = model.predict(XX_array)
  pred[pred < value] = 0
  pred[pred > value] = 1

  for element in range(YY_array.shape[0]): # Проходим по всем примерам в батче
    for word in range(YY_array.shape[1]):  # Проходим по каждому слову
      for category in range(YY_array.shape[2]): # Проходим по каждой категории в слове
        if pred[element][word][category] == YY_array[element][word][category]: # Если предсказанное значение совпадает с истинным:
          correct_list[category] += 1 # Отмечаем, что мы правильно предсказали класс объекта
        else:  # Если предсказанное значение НЕ совпадает с истенным:
          incorrect_list[category] += 1 # Отмечаем, что мы не правильно предсказали класс объекта
      
  # Итоговая точность для каждого класса = кол. 100% * правильных/(кол. неправильных + кол. правильных)
  for i in range(6):
   print("Сеть распознала категорию  '{}' с точностью в {}%".format(tags[i], round(100*correct_list[i]/(correct_list[i] + incorrect_list[i]), 2)))
  total = round(100*np.mean(correct_list/(correct_list + incorrect_list)),2) 
  print("Средняя точность {}%".format(total))

In [ ]:
print('Размер xTrain:', GENSIMtestX.shape)
print('Размер yTrain:', GENSIMtestY.shape)

Размер xTrain: (418, 256, 300)
Размер yTrain: (418, 256, 6)


In [ ]:
tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
# s1 Условия
# s2 Запреты
# s3 Стоимость (услуг и прочее)
# s4 Условия (относительно дат - дедлайны)
# s5 Штрафы
# s6 Адреса и местоположения

In [ ]:
recognizeSet(GENSIMtestX, GENSIMtestY, model_conv1d, tags, xLen, 0.5) # Проверяем conv1d архитектуру

Сеть распознала категорию  'S1' с точностью в 43.67%
Сеть распознала категорию  'S2' с точностью в 94.88%
Сеть распознала категорию  'S3' с точностью в 62.11%
Сеть распознала категорию  'S4' с точностью в 58.53%
Сеть распознала категорию  'S5' с точностью в 90.53%
Сеть распознала категорию  'S6' с точностью в 94.02%
Средняя точность 73.96%


# Выводы

На примере работы линейной модели Conv1D с текстами можно отметить повышение качества при условии добавления Dropout() и BatchNormalization() слоев. Изменение количества нейронов и batch_size показало, что слишком маленькие и слишком большие параметры не дают качественного результата. Наилучшая комбинация имеет значения на 128 нейронов в каждом слое и batch_size = 64.

[GENSIM Word2Vec документация](https://radimrehurek.com/gensim/models/word2vec.html)